In this notebook we aim to develop a protocole to evaluate OLAF pipelines:


To achieve this task , we will follow this steps:

- Select a corpus.
- Select and create relevent concepts from the corpus.
- Create several pipelines with different components and parameters.
- Run all the pipelines.
- Find concepts involved in complete triples (relation with no null source and destination concepts) for each pipeline.
- Etablish the matching percentage of found concepts compared to selected concepts on step 2.


In [89]:
import spacy
from typing import Set, List

from olaf import Pipeline
from olaf.data_container.relation_schema import Relation, Concept
from olaf.data_container.knowledge_representation_schema import KnowledgeRepresentation
from olaf.pipeline.pipeline_component.term_extraction import POSTermExtraction
from olaf.pipeline.pipeline_component.concept_relation_extraction import (
    CTsToConceptExtraction,
    CTsToRelationExtraction,
    SynonymRelationExtraction,
    SynonymConceptExtraction,
    AgglomerativeClusteringRelationExtraction,
    AgglomerativeClusteringConceptExtraction
)
from olaf.pipeline.pipeline_component.term_extraction.tfidf_term_extraction import (
    TFIDFTermExtraction,
)
from olaf.pipeline.pipeline_component.term_extraction.manual_candidate_terms import (
    ManualCandidateTermExtraction,
)
from olaf.repository.corpus_loader.text_corpus_loader import TextCorpusLoader

In [90]:
nlp = spacy.load("en_core_web_lg")

# Select Corpus

In [91]:
corpus_path = "GC10-DET_doc.txt"
corpus = TextCorpusLoader(corpus_path)._read_corpus()
corpus = [doc[:-1] for doc in corpus]
corpus

['Each defect type is described in detail, explaining how it appears on the steel strip surface and the reasons behind its occurrence:',
 '    Punching: In the production line of the strip, the steel strip needs to be punched according to the product specifications; mechanical failure may lead to unwanted punching, resulting in punching defects.',
 '    Welding line: When the strip is changed, it is necessary to weld the two coils of the strip, and the weld line is produced. Strictly speaking, this is not a defect, but it needs to be automatically detected and tracked to be circumvented in subsequent cuts.',
 '    Crescent gap: In the production of steel strip, cutting sometimes results in defects, just like half a circle.',
 '    Water spot: A water spot is produced by drying in production. Under different products and processes, the requirements for this defect are different. However, because the water spots are generally with low contrast, and are similar to other defects such as oi

# Select and create relevent concepts from the corpus.


In [92]:
"knfjre\n".rstrip("\n")

'knfjre'

In [93]:
expected_concepts = []
with open("concepts.txt", 'r') as f:
    lines = f.readlines()
    expected_concepts = [concept.rstrip("\n") for concept in lines]
    expected_concepts = [Concept(concept) for concept in expected_concepts]
    f.close()

print(expected_concepts)


[Punching, Welding line, Crescent Gap, Water spot, Oil spot, Silk spot, Inclusion, Rolled pit, Crease, Waist folding, metal surface defect, mechanical failure, drying, mechanical lubricant, temperature, pressure, work roll damage, tension roll damage, local yield, low-carbon]


# Testing concept ratio function

In [94]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util


def is_similar(concept_a : str, concept_b: str, nlp =nlp, threshold=.8):
    vector_a, vector_b = nlp(concept_a).vector, nlp(concept_b).vector
    return cosine_similarity([vector_a], [vector_b]) > threshold

def is_equal(concept_a : str, concept_b: str):
    return concept_a.lower() == concept_b.lower()

def hg_lm_similaritiry(concept_a : str, concept_b: str, model=SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2"), threshold=.8):
    embedding_a, embedding_b = model.encode(concept_a), model.encode(concept_b)
    return util.pytorch_cos_sim(embedding_a, embedding_b) > threshold

def get_concept_ratio(pipeline : Pipeline, expected_concepts : List[Concept], comparator = hg_lm_similaritiry, comparator_args:dict={}) -> tuple:
    """
    Calculate the ratio of expected and unexpected concepts in a given pipeline.

    Parameters
    ----------
    pipeline : Pipeline
        The pipeline object containing concepts.
    expected_concepts : List[Concept]
        A list of expected concepts.kwargs

    Returns
    -------
    Tuple[float, float]: A tuple containing:
        The percentage of expected concepts found in the pipeline.
        The percentage of unexpected concepts in the pipeline.
    """
    
    
    found_concepts = pipeline.kr.concepts
    found_concepts = [found_concept.label for found_concept in found_concepts]
    expected_concepts = [expected_concept.label for expected_concept in expected_concepts]
    expected_concept_occ = 0
    for expected_concept in expected_concepts:
        for found_concept in found_concepts:
            if comparator(expected_concept, found_concept, **comparator_args):
                expected_concept_occ += 1
                break 

  
    precision = expected_concept_occ/len(expected_concepts)
    recall = expected_concept_occ/len(found_concepts)
    f1 = 2*(precision * recall)/(precision+recall)
    return (precision, recall, f1)

In [95]:
from olaf.pipeline.pipeline_component.term_extraction.manual_candidate_terms import (
    ManualCandidateTermExtraction,
)
from olaf.pipeline.pipeline_component.concept_relation_extraction.candidate_terms_to_concepts import CTsToConceptExtraction

from olaf.pipeline.pipeline_component.concept_relation_extraction.candidate_terms_to_relations import CTsToRelationExtraction


# concept extraction component
concepts = [
    "defect type",
    "steel strip surface",
    "punching",
    "mechanical failure",
    "welding line",
    "coil",
    "weld line",
    "crescent gap",
    "cutting",
    "water spot",
    "drying",
    "oil spot",
    "mechanical lubricant",
    "silk spot",
    "plaque",
    "strip surface",
    "roller",
    "pressure",
    "inclusion",
    "metal surface",
    "spots",
    "fish scale shape",
    "block irregular distribution",
    "rolled pit",
    "bulges",
    "pits",
    "steel plate",
    "work roll",
    "tension roll",
    "damage",
    "crease",
    "fold",
    "uncoiling process",
    "waist folding",
    "deformation",
    "low-carbon"
]

relations = [
    "described",
    "explaining",
    "appears",
    "leads",
    "resulting",
    "changed",
    "produced",
    "drying",
    "caused",
    "affect",
    "appearing",
    "lies",
    "distributed",
    "accompanied",
    "showing",
    "pressed",
    "occurred",
    "circumvented",
    "detected",
    "tracked",
    "results",
    "like",
    "mainly",
    "uncoiling"
]

ct_concept_label = { concept : {concept} for concept in concepts}

manuel_concept_extraction = ManualCandidateTermExtraction(
    ct_label_strings_map=ct_concept_label
)

concept_extraction = CTsToConceptExtraction(
)
# concept extraction component



relation_extraction = CTsToRelationExtraction()
pipelines = []
pipelines.append(
    Pipeline(
        spacy_model=nlp,
        pipeline_components=[
            manuel_concept_extraction,
            concept_extraction,
        ],
        corpus=list(nlp.pipe(corpus)),
    )
)

[2024-06-14 15:41:49,263] [WARNING] [candidate_terms_to_relations] [_check_parameters] [No value given for concept_max_distance parameter, default will be set to 5.]


In [96]:
current_pipeline = pipelines[-1]
current_pipeline.run()

get_concept_ratio(current_pipeline, expected_concepts)

(0.8, 0.45714285714285713, 0.5818181818181818)

In [97]:
def debug_get_concept_ratio(pipeline : Pipeline, expected_concepts : List[Concept], comparator = hg_lm_similaritiry, comparator_args:dict={}) -> tuple:
    """
    Calculate the ratio of expected and unexpected concepts in a given pipeline.

    Parameters
    ----------
    pipeline : Pipeline
        The pipeline object containing concepts.
    expected_concepts : List[Concept]
        A list of expected concepts.kwargs

    Returns
    -------
    Tuple[float, float]: A tuple containing:
        The percentage of expected concepts found in the pipeline.
        The percentage of unexpected concepts in the pipeline.
    """
    
    
    found_concepts = pipeline.kr.concepts
    found_concepts = [found_concept.label for found_concept in found_concepts]
    expected_concepts = [expected_concept.label for expected_concept in expected_concepts]
    expected_concept_occ = 0
    for expected_concept in expected_concepts:
        print()
        print(f"{expected_concept} : ", end="")
        for found_concept in found_concepts:
            if comparator(expected_concept, found_concept, **comparator_args):
                print(f"{found_concept} ", end="")
                expected_concept_occ += 1
                break 

  
    precision = expected_concept_occ/len(expected_concepts)
    recall = expected_concept_occ/len(found_concepts)
    f1 = 2*(precision * recall)/(precision+recall)
    return (precision, recall, f1)



current_pipeline = pipelines[-1]
debug_get_concept_ratio(current_pipeline, expected_concepts)


Punching : punching 
Welding line : weld line 
Crescent Gap : crescent gap 
Water spot : water spot 
Oil spot : oil spot 
Silk spot : silk spot 
Inclusion : inclusion 
Rolled pit : rolled pit 
Crease : crease 
Waist folding : waist folding 
metal surface defect : 
mechanical failure : mechanical failure 
drying : drying 
mechanical lubricant : mechanical lubricant 
temperature : 
pressure : pressure 
work roll damage : 
tension roll damage : tension roll 
local yield : 
low-carbon : low-carbon 

(0.8, 0.45714285714285713, 0.5818181818181818)

In [98]:
debug_get_concept_ratio(current_pipeline, expected_concepts, comparator_args={"threshold":.7})


Punching : punching 
Welding line : weld line 
Crescent Gap : crescent gap 
Water spot : water spot 
Oil spot : oil spot 
Silk spot : silk spot 
Inclusion : inclusion 
Rolled pit : rolled pit 
Crease : crease 
Waist folding : waist folding 
metal surface defect : metal surface 
mechanical failure : mechanical failure 
drying : drying 
mechanical lubricant : mechanical lubricant 
temperature : 
pressure : pressure 
work roll damage : work roll 
tension roll damage : tension roll 
local yield : 
low-carbon : low-carbon 

(0.9, 0.5142857142857142, 0.6545454545454545)

In [99]:
debug_get_concept_ratio(current_pipeline, expected_concepts, comparator=is_equal)



Punching : punching 
Welding line : welding line 
Crescent Gap : crescent gap 
Water spot : water spot 
Oil spot : oil spot 
Silk spot : silk spot 
Inclusion : inclusion 
Rolled pit : rolled pit 
Crease : crease 
Waist folding : waist folding 
metal surface defect : 
mechanical failure : mechanical failure 
drying : drying 
mechanical lubricant : mechanical lubricant 
temperature : 
pressure : pressure 
work roll damage : 
tension roll damage : 
local yield : 
low-carbon : low-carbon 

(0.75, 0.42857142857142855, 0.5454545454545454)

In [100]:
debug_get_concept_ratio(current_pipeline, expected_concepts, comparator_args={"threshold":.5})


Punching : punching 
Welding line : weld line 
Crescent Gap : crescent gap 
Water spot : oil spot 
Oil spot : oil spot 
Silk spot : oil spot 
Inclusion : inclusion 
Rolled pit : rolled pit 
Crease : crease 
Waist folding : waist folding 
metal surface defect : steel strip surface 
mechanical failure : mechanical failure 
drying : drying 
mechanical lubricant : mechanical lubricant 
temperature : 
pressure : pressure 
work roll damage : damage 
tension roll damage : tension roll 
local yield : 
low-carbon : low-carbon 

(0.9, 0.5142857142857142, 0.6545454545454545)

## Optimze the similarity threshold


In [101]:
print(get_concept_ratio(current_pipeline, expected_concepts)) # default threshold is 0.8
print(get_concept_ratio(current_pipeline, expected_concepts, comparator_args={"threshold": 0.7}))
print(get_concept_ratio(current_pipeline, expected_concepts, comparator_args={"threshold": 0.6}))
print(get_concept_ratio(current_pipeline, expected_concepts, comparator_args={"threshold": 0.5}))

comparator_args={"threshold": 0.7}

(0.8, 0.45714285714285713, 0.5818181818181818)
(0.9, 0.5142857142857142, 0.6545454545454545)
(0.9, 0.5142857142857142, 0.6545454545454545)
(0.9, 0.5142857142857142, 0.6545454545454545)


#  a grid search algorithm for pipelines

In [102]:
class GridSearch:
    def __init__(self) -> None:
        pass

# Usefull function

In [103]:
def display_concept(kr: KnowledgeRepresentation) -> None:
    print("Concepts in KR:")
    for concept in kr.concepts:
        print(concept.label)


def display_relation(kr: KnowledgeRepresentation) -> None:
    print("Relations in KR:")
    for relation in kr.relations:
        if (
            relation.source_concept is not None
            or relation.destination_concept is not None
        ):
            print(
                (
                    relation.source_concept.label,
                    relation.label,
                    relation.destination_concept.label,
                )
            )

# Creating pipelines

In [104]:
from olaf.pipeline.pipeline_component.term_extraction import (
    ManualCandidateTermExtraction,
    POSTermExtraction,
    TFIDFTermExtraction,
    CvalueTermExtraction
)

from olaf.pipeline.pipeline_component.concept_relation_extraction import (
    CTsToConceptExtraction,
    SynonymConceptExtraction,
    AgglomerativeClusteringConceptExtraction
)


## Manuelle Term  Extraction

In [133]:
manual_pipelines = []

### Manuelle Term  Extraction and Candidat To Concept Extraction

In [106]:
# concept extraction component
concepts = [
    "defect type",
    "steel strip surface",
    "punching",
    "mechanical failure",
    "welding line",
    "coil",
    "weld line",
    "crescent gap",
    "cutting",
    "water spot",
    "drying",
    "oil spot",
    "mechanical lubricant",
    "silk spot",
    "plaque",
    "strip surface",
    "roller",
    "pressure",
    "inclusion",
    "metal surface",
    "spots",
    "fish scale shape",
    "block irregular distribution",
    "rolled pit",
    "bulges",
    "pits",
    "steel plate",
    "work roll",
    "tension roll",
    "damage",
    "crease",
    "fold",
    "uncoiling process",
    "waist folding",
    "deformation",
    "low-carbon"
]

ct_concept_label = { concept : {concept} for concept in concepts}


manual_pipelines.append(
    Pipeline(
        spacy_model=nlp,
        pipeline_components=[
            ManualCandidateTermExtraction(
                ct_label_strings_map=ct_concept_label
            ),
            CTsToConceptExtraction(),
        ],
        corpus=list(nlp.pipe(corpus)),
    )
)
current_pipeline = manual_pipelines[-1]
current_pipeline.run()


get_concept_ratio(current_pipeline, expected_concepts, comparator_args=comparator_args)

(0.9, 0.5142857142857142, 0.6545454545454545)

### Manuelle Term  Extraction and Synonym Concept Extraction

In [107]:

manual_pipelines.append(
    Pipeline(
        spacy_model=nlp,
        pipeline_components=[
            ManualCandidateTermExtraction(
                ct_label_strings_map=ct_concept_label
            ),
            SynonymConceptExtraction(),
        ],
        corpus=list(nlp.pipe(corpus)),
    )
)
current_pipeline = manual_pipelines[-1]
current_pipeline.run()


get_concept_ratio(current_pipeline, expected_concepts, comparator_args=comparator_args)

(0.9, 0.5142857142857142, 0.6545454545454545)

### Manuelle Term  Extraction and Agglomerative Clustering Concept Extraction

In [134]:

manual_pipelines.append(
    Pipeline(
        spacy_model=nlp,
        pipeline_components=[
            ManualCandidateTermExtraction(
                ct_label_strings_map=ct_concept_label
            ),
            AgglomerativeClusteringConceptExtraction(
                distance_threshold=.3
            ),
        ],
        corpus=list(nlp.pipe(corpus)),
    )
)
current_pipeline = manual_pipelines[-1]
current_pipeline.run()


# get_concept_ratio(current_pipeline, expected_concepts, comparator_args=comparator_args)

[2024-06-14 17:31:18,867] [WARNING] [agglomerative_clustering_concept_extraction] [_check_parameters] [No value given for embedding_model parameter, default will be set to all-mpnet-base-v2.]
[2024-06-14 17:31:18,868] [WARNING] [agglomerative_clustering_concept_extraction] [_check_parameters] [No value given for metric option, default will be set to cosine.]


OutOfMemoryError: CUDA out of memory. Tried to allocate 90.00 MiB. GPU 0 has a total capacty of 1.96 GiB of which 65.88 MiB is free. Including non-PyTorch memory, this process has 1.31 GiB memory in use. Of the allocated memory 1.19 GiB is allocated by PyTorch, and 84.53 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## POS tag Term Extraction

In [118]:
postag_pipelines = []


### POS tag Term  extraction and Candidat To Concept Extraction

In [117]:
postag_pipelines.append(
    Pipeline(
        spacy_model=nlp,
        pipeline_components=[
            POSTermExtraction(
                pos_selection=["NOUN"]
            ),
            CTsToConceptExtraction()
        ],
        corpus=list(nlp.pipe(corpus)),
    )
)
current_pipeline = postag_pipelines[-1]
current_pipeline.run()


get_concept_ratio(current_pipeline, expected_concepts, comparator_args=comparator_args)

[2024-06-14 17:13:50,508] [WARNING] [pos_term_extraction] [__init__] [No preprocessing function provided for spans. Using the default one.]
[2024-06-14 17:13:50,509] [WARNING] [pos_term_extraction] [_check_parameters] [POS term extraction token sequence attribute not set by the user.
               By default the system will use the entire content of the document.]


(0.6, 0.1643835616438356, 0.25806451612903225)

In [119]:
debug_get_concept_ratio(current_pipeline, expected_concepts, comparator_args=comparator_args)


Punching : punching 
Welding line : 
Crescent Gap : gap 
Water spot : 
Oil spot : 
Silk spot : silk 
Inclusion : inclusions 
Rolled pit : pit 
Crease : crease 
Waist folding : fold 
metal surface defect : 
mechanical failure : 
drying : 
mechanical lubricant : lubricant 
temperature : temperature 
pressure : pressure 
work roll damage : 
tension roll damage : 
local yield : yield 
low-carbon : carbon 

(0.6, 0.1643835616438356, 0.25806451612903225)

### POS tag Term  extraction and Synonym Concept Extraction

In [121]:
postag_pipelines.append(
    Pipeline(
        spacy_model=nlp,
        pipeline_components=[
            POSTermExtraction(
                pos_selection=["NOUN"]
            ),
            SynonymConceptExtraction()
        ],
        corpus=list(nlp.pipe(corpus)),
    )
)
current_pipeline = postag_pipelines[-1]
current_pipeline.run()


get_concept_ratio(current_pipeline, expected_concepts, comparator_args=comparator_args)

[2024-06-14 17:17:15,197] [WARNING] [pos_term_extraction] [__init__] [No preprocessing function provided for spans. Using the default one.]
[2024-06-14 17:17:15,199] [WARNING] [pos_term_extraction] [_check_parameters] [POS term extraction token sequence attribute not set by the user.
               By default the system will use the entire content of the document.]


(0.6, 0.1643835616438356, 0.25806451612903225)

### POS tag Term  extraction and Agglomerative clustering Extraction

In [122]:
postag_pipelines.append(
    Pipeline(
        spacy_model=nlp,
        pipeline_components=[
            POSTermExtraction(
                pos_selection=["NOUN"]
            ),
            AgglomerativeClusteringConceptExtraction(
                distance_threshold=.3
            )
        ],
        corpus=list(nlp.pipe(corpus)),
    )
)



get_concept_ratio(current_pipeline, expected_concepts, comparator_args=comparator_args)

[2024-06-14 17:17:31,199] [WARNING] [pos_term_extraction] [__init__] [No preprocessing function provided for spans. Using the default one.]
[2024-06-14 17:17:31,200] [WARNING] [pos_term_extraction] [_check_parameters] [POS term extraction token sequence attribute not set by the user.
               By default the system will use the entire content of the document.]
[2024-06-14 17:17:31,202] [WARNING] [agglomerative_clustering_concept_extraction] [_check_parameters] [No value given for embedding_model parameter, default will be set to all-mpnet-base-v2.]
[2024-06-14 17:17:31,203] [WARNING] [agglomerative_clustering_concept_extraction] [_check_parameters] [No value given for metric option, default will be set to cosine.]


(0.6, 0.1643835616438356, 0.25806451612903225)

## TFIDF  Term Extraction

In [126]:
tfidf_pipelines = []

### TFIDF Term Extraction and Candidat To Concept Extraction

In [130]:
tfidf_pipelines.append(
    Pipeline(
        spacy_model=nlp,
        pipeline_components=[
            TFIDFTermExtraction(
                max_term_token_length=3,
                candidate_term_threshold=.1
            ),
            CTsToConceptExtraction()
        ],
        corpus=list(nlp.pipe(corpus)),
    )
)
current_pipeline = tfidf_pipelines[-1]
current_pipeline.run()


get_concept_ratio(current_pipeline, expected_concepts, comparator_args=comparator_args)

[2024-06-14 17:22:14,528] [WARNING] [tfidf_term_extraction] [_check_parameters] [Selected token sequence document attribute not set by the user.
                By default the system will use the entire content of the document.]
/home/oumar/Bureau/ontology-learning/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(0.4, 0.034334763948497854, 0.06324110671936758)

In [131]:
debug_get_concept_ratio(current_pipeline, expected_concepts, comparator_args=comparator_args)


Punching : punching : in 
Welding line : 
Crescent Gap : crescent gap : 
Water spot : water spot 
Oil spot : oil spot : 
Silk spot : 
Inclusion : inclusion 
Rolled pit : 
Crease : crease 
Waist folding : 
metal surface defect : 
mechanical failure : mechanical failure may 
drying : 
mechanical lubricant : of mechanical lubricant 
temperature : 
pressure : 
work roll damage : 
tension roll damage : 
local yield : 
low-carbon : 

(0.4, 0.034334763948497854, 0.06324110671936758)

In [132]:
display_concept(current_pipeline.kr)

Concepts in KR:
appears on the
affect
punching : in
strip , the
in defects ,
roll
described in detail
strip surface and
detail ,
punched according to
and
type
defects .
appearance
crease
a circle
half
defects , just
in detail ,
according to the
the appearance
, explaining
of mechanical lubricant
described
in punching defects
surface and the
;
the production line
, just like
failure may
circle
: an
product specifications ;
circle .
or
detail , explaining
punched
and the reasons
little
mechanical lubricant ,
detail
the steel
appearance of
an oil
explaining how
, resulting in
: in
to unwanted
failure may lead
to unwanted punching
the defect
crescent gap :
just like half
, the steel
crescent gap
in punching
unwanted punching ,
lubricant , which
explaining
, explaining how
product .
the reasons behind
to the product
be punched according
contamination
like half a
explaining how it
gap :
is usually
how it
gap : in
how
how it appears
oil spot :
line of
may lead to
, which will
half a circle
it

### TFIDF Term Extraction and Synonym Concept Extraction

In [ ]:
postag_pipelines.append(
    Pipeline(
        spacy_model=nlp,
        pipeline_components=[
            TFIDFTermExtraction(
                max_term_token_length=3,
                candidate_term_threshold=.2
            ),
            SynonymConceptExtraction()
        ],
        corpus=list(nlp.pipe(corpus)),
    )
)
current_pipeline = postag_pipelines[-1]
current_pipeline.run()


get_concept_ratio(current_pipeline, expected_concepts, comparator_args=comparator_args)

[2024-06-14 11:54:54,452] [WARNING] [pos_term_extraction] [__init__] [No preprocessing function provided for spans. Using the default one.]
[2024-06-14 11:54:54,454] [WARNING] [pos_term_extraction] [_check_parameters] [POS term extraction token sequence attribute not set by the user.
               By default the system will use the entire content of the document.]


ZeroDivisionError: division by zero

### TFIDF Term Extraction and Agglomerative clustering Concept Extraction

In [ ]:
postag_pipelines.append(
    Pipeline(
        spacy_model=nlp,
        pipeline_components=[
            TFIDFTermExtraction(
                max_term_token_length=3,
                candidate_term_threshold=.2
            ),
            AgglomerativeClusteringConceptExtraction(
                distance_threshold=.3
            )
        ],
        corpus=list(nlp.pipe(corpus)),
    )
)
current_pipeline = postag_pipelines[-1]
current_pipeline.run()


get_concept_ratio(current_pipeline, expected_concepts, comparator_args=comparator_args)

[2024-06-14 12:35:13,587] [WARNING] [tfidf_term_extraction] [_check_parameters] [Selected token sequence document attribute not set by the user.
                By default the system will use the entire content of the document.]
/home/oumar/Bureau/ontology-learning/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(0.1, 0.5, 0.16666666666666669)